# Audio Classification with CNN under Cross-Validation Experiment
Try to analysis the data with,
+ accuracy
+ precision
+ recall
  
both of training step and validation stage.
Meanwhile, try to use the plot-box to demonstrates the data.

# The Cross-Validation under Multi-Classification
`This explaination comes from the Chat-GPT4o.`
Calculating precision and recall for a multi-class classification algorithm involves considering each class separately and then aggregating the results. Here's how you can do it step by step:

### 1. Definitions

- **Precision** for a class $C_i$: The ratio of true positive predictions for class $C_i$ to the total number of instances predicted as $C_i$ (i.e., true positives + false positives).
  
  $$
  \text{Precision}(C_i) = \frac{\text{True Positives}(C_i)}{\text{True Positives}(C_i) + \text{False Positives}(C_i)}
  $$

- **Recall** for a class $C_i $: The ratio of true positive predictions for class $ C_i $ to the total number of instances that actually belong to $ C_i $ (i.e., true positives + false negatives).

  $$
  \text{Recall}(C_i) = \frac{\text{True Positives}(C_i)}{\text{True Positives}(C_i) + \text{False Negatives}(C_i)}
  $$

### 2. Confusion Matrix

For a multi-class classification problem, you typically generate a confusion matrix, where each row represents the actual class, and each column represents the predicted class. The element at position $ (i, j) $ in the confusion matrix represents the number of instances of class $ i $ that were predicted as class $ j $.

### 3. Calculation for Each Class

- **True Positives (TP)**: The diagonal element for the class $ C_i $ in the confusion matrix, i.e., $ \text{TP}(C_i) = \text{Confusion Matrix}[i][i] $.
- **False Positives (FP)**: The sum of the corresponding column $ i $ minus the true positives, i.e., $ \text{FP}(C_i) = \sum_j \text{Confusion Matrix}[j][i] - \text{Confusion Matrix}[i][i] $.
- **False Negatives (FN)**: The sum of the corresponding row $ i $ minus the true positives, i.e., $ \text{FN}(C_i) = \sum_j \text{Confusion Matrix}[i][j] - \text{Confusion Matrix}[i][i] $.

### 4. Precision and Recall for Each Class

Using the definitions and the components from the confusion matrix, you can calculate precision and recall for each class $ C_i $:

$$
\text{Precision}(C_i) = \frac{\text{TP}(C_i)}{\text{TP}(C_i) + \text{FP}(C_i)}
$$

$$
\text{Recall}(C_i) = \frac{\text{TP}(C_i)}{\text{TP}(C_i) + \text{FN}(C_i)}
$$

### 5. Aggregating Results

To get overall metrics, you can use macro-averaging or micro-averaging:

- **Macro-Averaging**: Calculate the metrics for each class independently and then take the average.

  $$
  \text{Macro Precision} = \frac{1}{N} \sum_{i=1}^{N} \text{Precision}(C_i)
  $$

  $$
  \text{Macro Recall} = \frac{1}{N} \sum_{i=1}^{N} \text{Recall}(C_i)
  $$

- **Micro-Averaging**: Aggregate the contributions of all classes to calculate the metrics. Essentially, you sum up the true positives, false positives, and false negatives across all classes and then calculate precision and recall.

  $$
  \text{Micro Precision} = \frac{\sum_{i=1}^{N} \text{TP}(C_i)}{\sum_{i=1}^{N} (\text{TP}(C_i) + \text{FP}(C_i))}
  $$

  $$
  \text{Micro Recall} = \frac{\sum_{i=1}^{N} \text{TP}(C_i)}{\sum_{i=1}^{N} (\text{TP}(C_i) + \text{FN}(C_i))}
  $$

### Example

Let's consider a simple example with three classes A, B, and C, and the following confusion matrix:

$$
\begin{array}{c|ccc}
    & \text{Pred A} & \text{Pred B} & \text{Pred C} \\
    \hline
    \text{Actual A} & 40 & 5 & 5 \\
    \text{Actual B} & 3 & 50 & 2 \\
    \text{Actual C} & 2 & 4 & 45 \\
\end{array}
$$

For class A:

- $\text{TP}(A) = 40$
- $\text{FP}(A) = 3 + 2 = 5$
- $\text{FN}(A) = 5 + 5 = 10$

$$
\text{Precision}(A) = \frac{40}{40 + 5} = \frac{40}{45} \approx 0.89
$$

$$
\text{Recall}(A) = \frac{40}{40 + 10} = \frac{40}{50} = 0.8
$$

You can calculate precision and recall similarly for classes B and C and then aggregate the results using macro-averaging or micro-averaging as needed.